## Benchmark

In [1]:
import mikan
import numpy as np
import SimpleITK as sitk
import time
from medpy.metric import dc, hd, hd95, assd

## Load Data

In [ ]:
gt = sitk.ReadImage(rf"..\data\patients_26_ground_truth.nii.gz", sitk.sitkUInt8)
pred = sitk.ReadImage(rf"..\data\patients_26_segmentation.nii.gz", sitk.sitkUInt8)

gt_arr = sitk.GetArrayFromImage(gt)
pred_arr = sitk.GetArrayFromImage(pred)

# Downsample for faster
# If you're patient, you can comment out here and wait for the medpy to run for 30 minutes 😆
gt_arr = np.array(gt_arr[::2, ::2, ::2])
pred_arr =  np.array(pred_arr[::2, ::2, ::2])

### DSC

In [3]:
# mikan: DSC
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
dsc = evaluator.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t

# medpy: DSC
t = time.time()
for i in (1,2,3,4,5):
    dsc = dc(pred_arr == i, gt_arr == i)
medpy_costs = time.time() - t

print(f"DSC: {medpy_costs / mikan_costs :.2f}x faster")


DSC: 15.09x faster


### HD

In [4]:
# mikan: HD
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
hausdorff_distance = evaluator.labels([1,2,3,4,5]).metrics("HD")
mikan_costs = time.time() - t
print(f"Mikan has calculated Hausdorff distance and cost {mikan_costs:.2f} s.")
print(f"Let's waiting for medpy, be patient for a while...")

# medpy: HD
t = time.time()
for i in (1,2,3,4,5):
    hausdorff_distance = hd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
medpy_costs = time.time() - t

print(f"HD: {medpy_costs / mikan_costs :.2f}x faster")

Mikan has calculated Hausdorff distance and cost 0.78 s.
Let's waiting for medpy, be patient for a while...
HD: 68.78x faster


### HD/HD95/ASSD

In [5]:
# mikan: Distances
t = time.time()
evaluator = mikan.ArrayEvaluator(gt_arr, pred_arr, spacing=gt.GetSpacing())
mikan_distances = evaluator.labels([1,2,3,4,5]).metrics(["hd", "hd95", "assd"])
mikan_costs = time.time() - t
print(mikan_distances)
print(f"Mikan has calculated distance and cost {mikan_costs:.2f} s.")
print(f"Let's waiting for medpy, be patient for a while...")

# medpy: Distances
t = time.time()
medpy_results = {}
for i in (1,2,3,4,5):
    hd_ = hd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    hd95_ = hd95(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    assd_ = assd(pred_arr == i, gt_arr == i, voxelspacing=gt.GetSpacing()[::-1])
    medpy_results[i] = {
        "hd": hd_,
        "hd95": hd95_,
        "assd": assd_,
    }
medpy_costs = time.time() - t
print(medpy_results)
print(f"Distances: {medpy_costs / mikan_costs :.2f}x faster")

{'1': {'hd': 3.8066790103912354, 'hd95': 0.7410011291503906, 'assd': 0.24823972582817078}, '2': {'hd': 5.875893592834473, 'hd95': 0.8939142823219299, 'assd': 0.30189621448516846}, '3': {'hd': 7.1895952224731445, 'hd95': 0.7410049438476562, 'assd': 0.29092279076576233}, '4': {'hd': 55.0378303527832, 'hd95': 0.7410011291503906, 'assd': 0.2536499500274658}, '5': {'hd': 46.238975524902344, 'hd95': 0.7410125732421875, 'assd': 0.27448806166648865}}
Mikan has calculated distance and cost 0.85 s.
Let's waiting for medpy, be patient for a while...
{1: {'hd': 3.8066796490554484, 'hd95': 0.7409999966621399, 'assd': 0.24827345569665876}, 2: {'hd': 5.875891921349948, 'hd95': 0.893913303991663, 'assd': 0.3018870383121832}, 3: {'hd': 7.18959419428273, 'hd95': 0.7409999966621399, 'assd': 0.2909637348388582}, 4: {'hd': 55.03783156368531, 'hd95': 0.7409999966621399, 'assd': 0.25367831066385055}, 5: {'hd': 46.23897571237574, 'hd95': 0.7409999966621399, 'assd': 0.2745253765803193}}
Distances: 190.71x fast